## SELECT PACKAGE


## GET LIBRARIES

In [ ]:
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F

# data science libs
import numpy as np
import pandas as pd

# warning suppresion
import warnings; warnings.simplefilter('ignore')

### Business Context ###
There is a huge demand for used cars in the Indian Market today. As sales of new cars have slowed down in the recent past, the pre-owned car market has continued to grow over the past years and is larger than the new car market now.you work in a tech start-up that aims to find footholes in this market.

### Objective ###

As senior data scientist, you have to come up with a pricing model that can effectively predict the price of used cars and can help the business in devising profitable strategies using differential pricing. For example, if the business knows the market price, it will never sell anything below it.


## CONNECTION ##



In [ ]:
session = get_active_session()
session.sql_simplifier_enabled = True
snowflake_environment = session.sql('SELECT current_user(),current_version()').collect()

snowpark_version = VERSION
# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Loading Data ###
The data is in form of csv file will be loaded into snowflake tables and it will be used from there.

In [ ]:


ls @STAGE_RAWDATA

Load the csv data into snowpark dataframe and writing in snowflake table 

In [ ]:
df = session.read.option("INFER_SCHEMA", True).option("PARSE_HEADER", True).csv("@STAGE_RAWDATA/cars_data.csv")

In [ ]:
df.show()

df.write.save_as_table("Used_Cars_India", mode="overwrite", table_type="transient")

### Load the Table from snowflake table

In [ ]:
df = session.table('Used_Cars_India')

## DATA EXPLORATION 

Let explore do some exploratory analysis on the dataset

In [ ]:
#datatypes
df_sch = df.schema

df.dtypes

## DATA CLEANSING ACTIVITIES 

1. changing all the columns to uppercase 
2. Identifying the categorical and numeric cols 
3. Null handling 
4. uppercase the column values for cat columns 

In [ ]:
col_name_mapping = {col: col.upper().replace('"','') for col in df.columns}
df = df.rename(col_name_mapping)
df.show()

Converting the decimal to double type for null handling and smooth use in model prediction

In [ ]:
## get the category columns
cat_cols = []
for col in df.schema:
    if "String" in str(col.datatype):
        cat_cols.append(col.name)
print(cat_cols)


In [ ]:
## get the decimal columns
decimal_cols = []
for col in df.schema:
    if "Decimal" in str(col.datatype):
        decimal_cols.append(col.name)
#print(decimal_cols)
for col in decimal_cols:
    df = df.with_column(col,df[col].cast(DoubleType()))
df.show()



In [ ]:
df.select(F.array_unique_agg("TRANSMISSION")).show()
df.select(F.array_unique_agg("FUEL_TYPE")).show()
df.select(F.array_unique_agg("OWNER_TYPE")).show()
df.select(F.array_unique_agg("BRAND")).show()
df.select(F.array_unique_agg("MODEL")).show()
df.select(F.array_unique_agg("LOCATION")).show()
df.select(F.array_unique_agg("YEAR")).show()



In [ ]:
df.describe()